## Interacción con las capacidades del servidor MCP

Los **servidores MCP** pueden proporcionar una amplia variedad de capacidades, incluyendo las primitivas **tools**, **resources** y **prompts**.  
Como desarrollador de clientes, se interactúa con ellas a través del objeto **ClientSession** instanciado, y por lo general se utiliza el mismo flujo de trabajo de **dos pasos** para cada una de ellas: **descubrimiento** y **uso**.

Para cada una de estas capacidades, el descubrimiento se realiza mediante una solicitud **`<primitive>/list`**, que en el **SDK de Python** está encapsulada en los métodos **`list_<primitive>s()`**.  
Estas llamadas —si el servidor proporciona la capacidad correspondiente— devolverán una lista de las **herramientas**, **recursos** o **prompts** disponibles.

El uso de las capacidades resultantes requiere una llamada **`<primitive>/<verb>`**, donde el *verbo* depende del tipo de primitivo que se llama.  
En el SDK de Python, estas llamadas están encapsuladas con métodos **`<verb>_<primitive>()`**, como **`call_tool()`** para **`tool/call`**.

---

### Nota

El protocolo también permite que el **servidor** (y el **cliente**) envíen mensajes de **notificación**.  
Posteriormente, el receptor gestiona las notificaciones según su conveniencia.  
En las conexiones **HTTP Streamable**, estas notificaciones se enviarán como cualquier otro mensaje, según lo definido por el transporte, pero como una implementación **`JSONRPCNotification`** de la clase **`JSONRPCMessage`**.

---

### Buenas prácticas de desarrollo

Si bien los SDK MCP disponibles envuelven las distintas llamadas de solicitud en métodos que el cliente puede invocar directamente, a menudo es recomendable **envolver estos métodos en funciones propias**.  
Esto permite agregar:

- **Registros (logging)**  
- **Reintentos automáticos**  
- **Parámetros adicionales de método**  
- **Otras personalizaciones** a las llamadas al servidor  

Por ejemplo, se puede pasar cualquiera de los métodos **`list_<primitive>()`** al parámetro opcional **`cursor`** para manejar **paginación**.

---

## Herramientas

Las **herramientas (tools)** son el recurso más común que ofrecen los servidores MCP —y con razón—, ya que los **flujos de trabajo agénticos** suelen requerir herramientas para **ampliar las capacidades de un LLM básico**.

Una **herramienta** es simplemente una **función determinista** que un LLM puede decidir llamar si lo considera necesario.  
Naturalmente, esta función puede hacer **cualquier cosa que pueda codificarse**, desde calcular números hasta consultar el clima en una ubicación determinada.  
Los servidores MCP permiten a los clientes **descubrir y utilizar estas herramientas**.

---

### Descubrir herramientas con el SDK de Python

Para descubrir herramientas, el cliente simplemente llama a:

```python
self.session.list_tools()
````

Esta llamada devuelve un objeto **`response`** con una propiedad **`tools`**,
que contiene la lista de herramientas disponibles en el servidor conectado al cliente.

Cada herramienta incluye las siguientes propiedades:

* `name`
* `description`
* `inputSchema`
* `annotations`
* `model_config`

Puedes consultar la estructura actual de la clase **`Tool`** en el módulo de tipos del SDK de Python de MCP.

---

### Extender el uso de `list_tools()`

Llamar a **`list_tools()`** directamente puede ser suficiente para casos básicos,
pero si estás construyendo un cliente que se usará en **diferentes servidores**,
con distintas capacidades o en **diferentes aplicaciones**,
puede ser ventajoso **escribir una función propia** que incluya:

* Filtros o restricciones específicas
* Registro (logging) detallado
* Manejo de errores
* Cualquier otra funcionalidad que tus usuarios puedan necesitar

Por ejemplo, podrías implementar una función similar a la siguiente:


---

### `client.py`

```python
import logging
from typing import Any

logger = logging.getLogger(__name__)

class MCPClient:
    ...
    async def get_available_tools(self) -> list[dict[str, Any]]:
        if not self._connected:
            raise RuntimeError("Client not connected to a server")

        tools_result = await self._session.list_tools()

        if not tools_result.tools:
            logger.warning("No tools found on server")

        available_tools = [
            {
                "name": tool.name,
                "description": tool.description,
                "input_schema": tool.inputSchema,
            }
            for tool in tools_result.tools
        ]

        return available_tools
````

---

### `agent.py`

```python
async def main():
    await mcp_client.connect()

    available_tools = await mcp_client.get_available_tools()
    print(f"Available tools: {', '.join([tool['name'] for tool in available_tools])}")

    while True:
        ...
```


## Uso de herramientas en un cliente MCP

El método anterior es un **envoltorio sencillo** alrededor del método `list_tools()` del objeto `Session`, el cual a su vez envuelve la solicitud `tools/list`.  
Aunque es simple, el ejemplo muestra cómo implementar **pequeñas medidas de seguridad**, como verificar el estado de conexión del servidor, así como **registros más detallados**, como la comprobación de resultados vacíos de herramientas.

El método también convierte la lista de objetos de herramientas nativos de MCP en una **lista de diccionarios** con las claves `"name"`, `"description"` y `"input_schema"`.  
Esto garantiza **compatibilidad con la API de mensajes de Anthropic**.

Puedes extender este patrón para:

- Agregar **reintentos automáticos**  
- Traducir el formato de herramientas de MCP a otro formato compatible con otro LLM  
- O encapsular los objetos `Tool` en tus propias clases personalizadas  

En el ejemplo de la función `main()`, se agregó una **list comprehension** para construir una lista con los nombres de las herramientas disponibles y se imprimió para verificar el funcionamiento del código.

---

## Ejecutar herramientas: `use_tool()`

Una vez que tu cliente tiene herramientas disponibles, el siguiente paso natural es **usarlas**.  
Al igual que en el ejemplo de `get_available_tools()`, ahora se envolverá el método `call_tool()` del objeto `Session`, añadiendo **registro (logging)** y **manejo de los diferentes tipos de respuesta** posibles.

El método `call_tool()` devuelve un objeto **`CallToolResult`**, que contiene una lista de cualquier tipo de contenido definido en el SDK:

- **`TextContent`** — El resultado de la herramienta es texto, almacenado en la propiedad `text`.  
- **`ImageContent`** — El resultado es una imagen, almacenada como cadena **base64** en la propiedad `data`.  
- **`AudioContent`** — El resultado es audio, también codificado en **base64** en la propiedad `data`.  
- **`EmbeddedResource`** — La herramienta devuelve recursos embebidos, usualmente como contexto adicional o para almacenamiento en caché.  
  - Puede contener un `TextResourceContents` (texto de un archivo o configuración).  
  - O un `BlobResourceContents` (datos binarios en formato **base64**).

> **Importante:** las solicitudes de uso de herramientas devuelven **listas de contenidos**, y en algunos casos —por ejemplo, cuando se usa un `EmbeddedResource` como contexto adicional— esto debe esperarse y manejarse correctamente.

A continuación se muestra un ejemplo completo de cómo envolver `call_tool()` y devolver **representaciones en cadena** de cada tipo de contenido válido.

---

### `client.py`

```python
from typing import Any
from mcp.types import TextResourceContents

class MCPClient:
    ...
    async def use_tool(self, tool_name: str, arguments: dict[str, Any] | None = None) -> list[str]:
        if not self._connected:
            raise RuntimeError("Client not connected to a server")

        tool_call_result = await self._session.call_tool(name=tool_name, arguments=arguments)
        logger.debug(f"Calling tool {tool_name} with arguments {arguments}")

        results = []
        if tool_call_result.content:
            for content in tool_call_result.content:
                match content.type:
                    case "text":
                        results.append(content.text)
                    case "image" | "audio":
                        results.append(content.data)
                    case "resource":
                        if isinstance(content.resource, TextResourceContents):
                            results.append(content.resource.text)
                        else:
                            results.append(content.resource.blob)
        else:
            logger.warning(f"No content in tool call result for tool {tool_name}")

        return results
````

---

### Explicación del método

El método **`use_tool()`**, al igual que `list_tools()`, primero:

1. **Verifica la conexión** del cliente al servidor MCP.
2. Llama a **`call_tool()`**, pasando el nombre de la herramienta y sus argumentos.
3. Registra la acción con un mensaje de depuración (`logger.debug`).

Luego:

* Si la respuesta contiene contenido (`content`), itera sobre cada elemento y **evalúa su tipo** (`text`, `image`, `audio`, `resource`).
* Convierte cada tipo en una **representación de texto o base64** y la añade a la lista de resultados.
* Si la respuesta está vacía, genera una advertencia (`logger.warning`).
* Finalmente, **devuelve la lista de resultados**.

Este patrón permite **control total sobre el flujo de ejecución** y el manejo de resultados en las llamadas a herramientas.

---

### ⚠️ Advertencias importantes

* En algunos casos, **manejar los tipos de respuesta en el cliente** puede no ser lo ideal.
  Antes de hacerlo, considera si tus usuarios necesitan controlar cómo se gestionan los diferentes tipos de contenido.

* Cuando construyas **agentes que soporten MCP**, monitorea cuidadosamente:

  * El **rendimiento general** del agente.
  * La **eficiencia del llamado a herramientas**.

  A medida que se añaden más herramientas, el rendimiento tiende a disminuir debido a:

  * **Ambigüedad u overlap** entre descripciones de herramientas.
  * **Interfaces demasiado detalladas**, que aumentan la carga cognitiva del LLM para seleccionar la herramienta correcta.

---

## Integración con modelos de lenguaje

Si estás construyendo un cliente para una aplicación específica (por ejemplo, un **chat con IA**), también deberás **utilizar estas llamadas**.

Muchas APIs de LLM que soportan el llamado a herramientas tienen un parámetro opcional **`tools`**, que permite enviar fácilmente las herramientas disponibles al modelo.
Otros modelos requieren **incrustar la lista de nombres, descripciones y esquemas** en el *system prompt*.

En este ejemplo nos centramos en la **API de Claude de Anthropic**, la cual **sí acepta un parámetro `tools`**.
Este parámetro se añade a los mensajes del usuario enviados al modelo; el host analiza las respuestas buscando invocaciones de herramientas y, si las encuentra, construye un **mensaje `tool_result`** que se agrega al historial (`messages`) antes de generar el resultado final para el usuario.

Así, el modelo puede **acceder a las herramientas proporcionadas por el servidor MCP** y **llamarlas dinámicamente** cuando lo requiera.

In [ ]:
# agent.py
from pathlib import Path
import asyncio

mcp_client = MCPClient(
    name="calculator_server_connection",
    command="uv",
    server_args=[
        "--directory",
        str(Path(__file__).parent.resolve()),
        "run",
        "calculator_server.py",
    ],
)

print("Bienvenido a tu Asistente de IA. Escribe 'adiós' para salir.")

async def main():
    """Función principal asincrónica que ejecuta el asistente."""
    try:
        await mcp_client.connect()
        available_tools = await mcp_client.get_available_tools()
        print(
            f"Herramientas disponibles: {', '.join([tool['name'] for tool in available_tools])}"
        )

        while True:
            prompt = input("Tú: ")
            if prompt.lower() == "adiós":
                print("Asistente IA: ¡Hasta luego!")
                break

            # Construir la conversación iniciando con el mensaje del usuario
            conversation_messages = [{"role": "user", "content": prompt}]

            # Bucle de uso de herramientas: continúa hasta obtener una respuesta final de texto
            while True:
                # Obtener respuesta del modelo LLM
                current_response = anthropic_client.messages.create(
                    max_tokens=4096,
                    messages=conversation_messages,
                    model="claude-sonnet-4-0",
                    tools=available_tools,
                    tool_choice={"type": "auto"},
                )

                # Agregar mensaje del asistente a la conversación
                conversation_messages.append(
                    {"role": "assistant", "content": current_response.content}
                )

                # Verificar si es necesario usar herramientas
                if current_response.stop_reason == "tool_use":
                    # Extraer bloques de uso de herramientas
                    tool_use_blocks = [
                        block
                        for block in current_response.content
                        if block.type == "tool_use"
                    ]

                    # Ejecutar todas las herramientas y recopilar resultados
                    tool_results = []
                    for tool_use in tool_use_blocks:
                        print(f"Usando herramienta: {tool_use.name}")
                        tool_result = await mcp_client.use_tool(
                            tool_name=tool_use.name, arguments=tool_use.input
                        )
                        tool_results.append(
                            {
                                "type": "tool_result",
                                "tool_use_id": tool_use.id,
                                "content": "\n".join(tool_result),
                            }
                        )

                    # Agregar resultados de herramientas a la conversación
                    conversation_messages.append(
                        {"role": "user", "content": tool_results}
                    )

                    continue

                else:
                    # No se requieren herramientas, extraer la respuesta final en texto
                    text_blocks = [
                        content.text
                        for content in current_response.content
                        if hasattr(content, "text") and content.text.strip()
                    ]

                    if text_blocks:
                        print(f"Asistente: {text_blocks[0]}")
                    else:
                        print("Asistente: [No hay respuesta de texto disponible]")

                    break
    finally:
        await mcp_client.disconnect()


if __name__ == "__main__":
    asyncio.run(main())


Este código tiene varios cambios para manejar la obtención y el uso de las herramientas necesarias. Primero, durante la fase de inicialización, obtenemos todas las herramientas disponibles del servidor al que estamos conectados mediante `get_available_tools()`. Luego, transformamos la lista de objetos `Tool` en una lista de diccionarios simples para poder devolverla correctamente al modelo. Esto se hace al inicio del módulo para obtener la lista de herramientas solo una vez. Si esperas que el servidor o los servidores a los que te conectas cambien las herramientas que ofrecen mientras estás conectado, deberás obtener una nueva lista de herramientas disponibles en cada entrada del usuario al modelo o hacer que tu cliente escuche y maneje una notificación `list_changed` del servidor.

A continuación, actualizamos el código donde creamos el mensaje del usuario para el modelo, pasando la lista de herramientas al parámetro `tools`, y configurando `tool_choice` como un diccionario con la clave `"type"` establecida en `"auto"`. No es obligatorio hacerlo, ya que `"auto"` es la configuración predeterminada y permite que el modelo decida qué herramienta o herramientas usar para responder la consulta del usuario. También puedes establecerlo en `"any"`, donde usará cualquiera (pero al menos una) de las herramientas disponibles, o en un nombre de herramienta y tipo `"tool"` para forzar al modelo a usar una herramienta específica, o `"none"` para evitar que el modelo use cualquier herramienta disponible.

Después de esto, inspeccionamos el resultado e inicializamos un posible mensaje de uso de herramienta que será enviado de vuelta al modelo. Si la respuesta del mensaje tiene un `stop_reason` de `tool_use`, entonces sabemos que el modelo está esperando una respuesta con los resultados de la o las herramientas que está solicitando. Creamos bloques de mensajes de uso de herramienta a partir del contenido de la respuesta y luego ejecutamos todas las herramientas en un bucle. A partir de los resultados, construimos un diccionario con la clave `type` establecida en `"tool_use"`, `tool_use_id` establecida en el ID de la respuesta de uso de herramienta del modelo, y `content` con el resultado de la llamada a la herramienta. Todos estos se agregan luego a la lista `conversation_messages`. Una vez realizadas todas las llamadas a herramientas, los resultados y la pregunta original del usuario se envían de vuelta al LLM para su evaluación. Dado que potencialmente pueden realizarse múltiples llamadas a herramientas antes de devolver una respuesta al usuario (como preguntar al modelo cuánto es 5 * 3 + 7), envolvemos todo esto en un bucle `while`.

Si no tenemos ninguna respuesta de llamada a herramienta en el mensaje de uso de herramienta, significa que no se necesitaron herramientas o que hemos llegado al final de la fase de uso de herramientas. En cualquier caso, extraemos los bloques de texto de la última respuesta y los imprimimos si están disponibles. Finalmente, todo el proceso está envuelto en un bloque `try/finally` para asegurar que la conexión se desconecte correctamente cuando el usuario cierre la aplicación.

En esta sección, aprendiste sobre las herramientas proporcionadas por el servidor MCP, cómo descubrirlas en un servidor, cómo usarlas y cómo utilizarlas en el contexto de una aplicación de chat. En la siguiente sección, aprenderás sobre los recursos y cómo puedes usarlos en tu aplicación de manera similar a como lo hiciste con las herramientas.


# Recursos

Como aprendiste en el **Capítulo 2**, los **recursos MCP** pueden desempeñar muchos roles, algunos de los cuales aún no han sido completamente explorados por la comunidad, como servir de **caché para optimizar el tamaño de los *prompts***.

Básicamente, los recursos permiten que una **aplicación anfitriona** y el **LLM** con el que interactúa accedan a datos proporcionados por el servidor MCP.
Estos datos pueden ser registros de bases de datos, imágenes, archivos de texto, y más.

Los recursos admiten varias **acciones**:

---

### Acciones principales

* **resources/list** o **list_resources()**
  Solicita al servidor una lista de los recursos disponibles.

* **resources/templates/list** o **list_resource_templates()**
  Solicita una lista de **plantillas de URI de recursos** desde el servidor.
  Estas permiten construir dinámicamente la URI de un recurso en función de otra información que tengas.
  En el SDK de Python, estas cadenas se asemejan a **f-strings**, con secciones variables entre llaves `{}`.

* **resources/read** o **read_resource()**
  Accede al archivo proporcionado por el servidor, enviando los datos al LLM en formato **texto** o **blob**.

* **resources/subscribe** o **subscribe_resource()**
  Suscribe al cliente a las actualizaciones del recurso cuya URI se especifica en la llamada.

* **resources/unsubscribe** o **unsubscribe_resource()**
  Cancela la suscripción del cliente a las actualizaciones del recurso especificado mediante su URI.

---

> **Nota**
> Consulta el proyecto **tupac** de *Tim Kellogg* para ver una implementación minimalista de cliente que utiliza **recursos MCP como caché** para *prompting* eficiente.

- https://github.com/tkellogg/tupac/
---

Primero implementaremos **wrappers** para los dos métodos de listado.
Seguiremos el patrón establecido en la sección anterior con `get_available_tools()`: un **envoltorio ligero**, con algunas verificaciones básicas y *logging*, que puede servir de base para flujos de trabajo más complejos.


In [2]:
# client.py

from mcp.types import Resource, ResourceTemplate, BlobResourceContents, TextResourceContents

class MCPClient:
    ...
    async def get_available_resources(self) -> list[Resource]:
        if not self._connected:
            raise RuntimeError("Client not connected to a server")

        resources_result = await self._session.list_resources()
        if not resources_result.resources:
            logger.warning("No resources found on server")
        return resources_result.resources

    async def get_available_resource_templates(self) -> list[ResourceTemplate]:
        if not self._connected:
            raise RuntimeError("Client not connected to a server")

        resource_templates_result = await self._session.list_resource_templates()
        if not resource_templates_result.resources:
            logger.warning("No resource templates found on server")
        return resource_templates_result.resources

    async def get_resource(self, uri: str) -> list[BlobResourceContents | TextResourceContents]:
        if not self._connected:
            raise RuntimeError("Client not connected to a server")
        
        resource_read_result = await self._session.read_resource(uri=uri)
        if not resource_read_result.contents:
            logger.warning(f"No content read for resource URI {uri}")
        return resource_read_result.contents


Estos métodos son casi idénticos entre sí y al método `get_available_tools()`. Cada función:

1. Verifica que exista una conexión antes de realizar solicitudes al servidor.
2. Llama al método de listado apropiado (`list_resources()` o `list_resource_templates()`).
3. Comprueba si hay resultados y emite una advertencia si la lista está vacía.
4. Devuelve los resultados.

Al igual que `list_tools()`, estos métodos aceptan un parámetro opcional `cursor` para **paginación de resultados**.

---

> **Nota**
> Los recursos y las plantillas de recursos tienen casi el mismo conjunto de propiedades: `uri`, `name`, `description`, `mimeType`, `size`, `annotations` y `model_config`.
> En las plantillas de recursos, en lugar de `uri`, se usa la propiedad `uriTemplate`.

---

El método `get_resource()` sigue el mismo patrón que `get_available_tools()`:
no transforma los objetos MCP `BlobResourceContents` y `TextResourceContents`, sino que los devuelve tal cual, proporcionando información útil sobre el tipo de contenido y manteniendo la filosofía de **procesamiento mínimo**.

---

El protocolo MCP también permite que los clientes se **suscriban a notificaciones de cambios** en los recursos.
Actualmente, en el SDK de Python, la gestión de notificaciones está **parcialmente implementada** y no existe forma de registrar manejadores específicos.
En cambio, en el SDK de TypeScript sí hay soporte para callbacks en `ClientSession`, que manejan todas las entradas, incluidas solicitudes, respuestas y notificaciones.

---

Hasta ahora aprendiste:

1. Cómo obtener una lista de **recursos disponibles** y **plantillas de recursos**.
2. Cómo leer un recurso usando su URI.

---

### Uso en una aplicación anfitriona

Los recursos se usan principalmente para **proporcionar contexto adicional** a las consultas de los usuarios, por ejemplo, agregando un script Bash a una pregunta sobre dicho script.

En nuestro ejemplo de chatbot minimalista:

* Buscamos la clave `"context:"` en el *prompt* del usuario.
* Cuando se detecta, el contenido posterior se utiliza para seleccionar un recurso específico.
* Ese recurso se agrega como **contexto** a la consulta original del usuario.

Cuando la aplicación inicia o el usuario escribe `"refresh"`, la aplicación:

1. Obtiene la lista actual de recursos del servidor.
2. Los muestra al usuario.

---

El ejemplo de código está dividido en dos archivos: `client.py` y `agent.py`, para **reducir la complejidad** y mejorar la legibilidad.

En `agent.py` ahora se encuentra la clase **Agent**, que incluye:

* `run()` → anteriormente `main()`.
* Funciones utilitarias para recursos:

  * `extract_resource_name()` → extrae el nombre del recurso del *prompt* del usuario.
  * `display_resources()` → muestra nombres y descripciones de los recursos disponibles.
  * `refresh_resources()` → actualiza y devuelve los recursos disponibles como un diccionario.

```



In [7]:
# agent.py

...
class Agent:
    def __init__(self, mcp_client: MCPClient, anthropic_client: Anthropic):
        self.mcp_client = mcp_client
        self.anthropic_client = anthropic_client
        self.available_resources = {}

    async def _select_resources(self, prompt: str) -> list[str]:
        """Usar el LLM para seleccionar inteligentemente recursos relevantes."""
        if not self.available_resources:
            return []

        resource_descriptions = {
            name: resource.description or f"Resource: {name}"
            for name, resource in self.available_resources.items()
        }

        selection_prompt = f"""
                        Dada esta pregunta del usuario: "{prompt}"
                        
                        Y estos recursos disponibles:
                        {json.dumps(resource_descriptions, indent=2)}
                        
                        ¿Qué recursos (si los hay) serían útiles para responder la pregunta del usuario?
                        Devuelve un arreglo JSON con los nombres de los recursos, o un arreglo vacío si no se necesitan recursos.
                        Solo incluye recursos que sean directamente relevantes.
                        
                        Ejemplo: ["math-constants"] o []
                        """

        try:
            response = self.anthropic_client.messages.create(
                max_tokens=200,
                messages=[{"role": "user", "content": selection_prompt}],
                model="claude-sonnet-4-0",
            )

            response_text = response.content[0].text.strip()
            # Extraer JSON de la respuesta (manejar caso donde el LLM añade explicación)
            if "[" in response_text and "]" in response_text:
                start = response_text.find("[")
                end = response_text.rfind("]") + 1
                json_part = response_text[start:end]
                selected_resources = json.loads(json_part)
                return [r for r in selected_resources if r in self.available_resources]

        except Exception as e:
            logger.warning(f"No se pudieron seleccionar recursos con el LLM: {e}")

        return []

    async def _load_selected_resources(
        self, resource_names: list[str]
    ) -> list[dict[str, Any]]:
        """Cargar los recursos especificados."""
        context_messages = []

        for resource_name in resource_names:
            if resource_name in self.available_resources:
                print(f"LLM seleccionó el recurso: {resource_name}")
                try:
                    resource = self.available_resources[resource_name]
                    resource_contents = await self.mcp_client.get_resource(
                        uri=resource.uri
                    )
                    for content in resource_contents:
                        if isinstance(content, TextResourceContents):
                            context_messages.append(
                                {
                                    "type": "text",
                                    "text": f"[Resource: {resource_name}]\n{content.text}",
                                }
                            )
                        elif content.mimeType in [
                            "image/jpeg",
                            "image/png",
                            "image/gif",
                            "image/webp",
                        ]:  # imagen codificada en base64
                            context_messages.append(
                                {
                                    "type": "image",
                                    "source": {
                                        "type": "base64",
                                        "media_type": content.mimeType,
                                        "data": content.blob,
                                    },
                                }
                            )
                        else:
                            print(
                                f"ADVERTENCIA: No se puede procesar mimeType {resource_contents.mimeType} para el recurso {resource_name}"
                            )
                except Exception as e:
                    print(f"Error cargando recurso {resource_name}: {e}")

        return context_messages

    async def _refresh_resources(self) -> None:
        available_resources = await self.mcp_client.get_available_resources()
        self.available_resources = {
            resource.name: resource for resource in available_resources
        }

    async def run(self):
        try:
            print(
                "Bienvenido a tu Asistente de IA. Escribe 'goodbye' para salir o 'refresh' para recargar y mostrar los recursos disponibles."
            )
            await self.mcp_client.connect()
            available_tools = await self.mcp_client.get_available_tools()
            await self._refresh_resources()

            while True:
                prompt = input("You: ")

                if prompt.lower() == "goodbye":
                    print("AI Assistant: ¡Adiós!")
                    break

                if prompt.lower() == "refresh":
                    await self._refresh_resources()
                    continue

                selected_resource_names = await self._select_resources(prompt)
                context_messages = await self._load_selected_resources(
                    selected_resource_names
                )

                # Construir conversación con mensaje inicial y contexto
                user_content = [{"type": "text", "text": prompt}]
                if context_messages:
                    user_content.extend(context_messages)

                conversation_messages = [{"role": "user", "content": user_content}]

                # Bucle de uso de herramientas hasta obtener respuesta final de texto
                while True:
                    current_response = anthropic_client.messages.create(
                        max_tokens=4096,
                        messages=conversation_messages,
                        model="claude-sonnet-4-0",
                        tools=available_tools,
                        tool_choice={"type": "auto"},
                    )

                    conversation_messages.append(
                        {"role": "assistant", "content": current_response.content}
                    )

                    if current_response.stop_reason == "tool_use":
                        tool_use_blocks = [
                            block
                            for block in current_response.content
                            if block.type == "tool_use"
                        ]

                        print(f"Ejecutando {len(tool_use_blocks)} herramienta(s)...")

                        tool_results = []
                        for tool_use in tool_use_blocks:
                            print(f"Usando herramienta: {tool_use.name}")
                            tool_result = await self.mcp_client.use_tool(
                                tool_name=tool_use.name, arguments=tool_use.input
                            )
                            tool_results.append(
                                {
                                    "type": "tool_result",
                                    "tool_use_id": tool_use.id,
                                    "content": "\n".join(tool_result),
                                }
                            )

                        conversation_messages.append(
                            {"role": "user", "content": tool_results}
                        )
                        continue
                    else:
                        text_blocks = [
                            content.text
                            for content in current_response.content
                            if hasattr(content, "text") and content.text.strip()
                        ]

                        if text_blocks:
                            print(f"Assistant: {text_blocks[0]}")
                        else:
                            print("Assistant: [No hay respuesta de texto disponible]")

                        break
        finally:
            await self.mcp_client.disconnect()


NameError: name 'Anthropic' is not defined

In [5]:
if __name__ == "__main__":
    mcp_client = MCPClient(
        name="calculator_server_connection",
        command="uv",
        server_args=[
            "--directory",
            str(Path(__file__).parent.parent.resolve()),
            "run",
            "calculator_server.py",
        ],
    )
    agent = Agent(mcp_client, anthropic_client)
    asyncio.run(agent.run())


NameError: name 'Path' is not defined

Esta integración específica comenzó igual que la integración que hiciste en la sección de Herramientas: conectarse al servidor MCP a través del cliente, descubrir los primitivos disponibles, como recursos en este caso, y cargar esos primitivos en memoria. Añadimos dos nuevas funciones utilitarias al agente:

- `_select_resources()`: Crea un diccionario nombre: descripción de los recursos disponibles y luego usa el LLM para seleccionar cualquier recurso relevante dado el prompt del usuario.

- `_load_selected_resources()`: Toma una lista de nombres de recursos de `_select_resources()` y, para los nombres válidos, usa el cliente para obtener los contenidos del recurso. Si esos contenidos son texto, creamos un bloque de contenido con el texto. Si es un blob, verificamos que el `mimeType` coincida con alguno de los tipos de imagen soportados por la API del modelo de Anthropic y, si es así, construimos un bloque de contenido con esos datos y lo añadimos a la lista final de mensajes que se enviarán al llamador.

- `_refresh_resources()`: Actualiza los recursos disponibles y los almacena en el diccionario `available_resources` del objeto Agent.

En la función `run()`, actualizamos el mensaje de bienvenida para informar al usuario que un mensaje con solo la palabra `refresh` actualizará la lista de recursos. La lista de recursos se llena con los resultados del método `_refresh_resources()`. Luego, cuando el usuario envía un prompt, el LLM selecciona los recursos pertinentes mediante `_select_resources()`. Después, llamamos a `_load_selected_resources()` para añadir cualquier recurso seleccionado a la lista de `context_messages`. Los mensajes de contexto proporcionan contexto adicional al LLM, y los combinamos con el prompt del usuario en una lista. Esta lista de prompt del usuario y recursos seleccionados se envía al LLM, que podrá usar el contenido de los recursos para responder al prompt del usuario si es necesario.

> NOTA  
Al añadir contexto a un prompt del usuario, no deberías crear un mensaje de usuario separado como con el prompt original. En su lugar, convierte el mensaje del usuario a su forma completa: un diccionario anidado con la clave `type` y la clave `content`, que contiene una lista de diccionarios, y añade cualquier mensaje de contexto a esa lista.

Hay varias mejoras que puedes hacer en este ejemplo. Para ampliar tus conocimientos, intenta implementar soporte para listar y usar plantillas de recursos junto con recursos normales. Luego, intenta soportar múltiples archivos de contexto en lugar de solo uno, ya sea con parsing de texto regular o incluso con otra llamada a un LLM. También puedes optimizar: actualmente, nuestra aplicación hace una llamada adicional potencialmente costosa al LLM por cada entrada del usuario para seleccionar los recursos más relevantes para su consulta. ¿Cómo podrías hacer esto más eficiente?

> NOTA  
Como habrás notado, muchas de estas mejoras son prácticas típicas de ingeniería de software y no son específicas de IA generativa. Aunque la ingeniería de IA se está convirtiendo en una disciplina propia, sigue siendo ingeniería de software en esencia, y las prácticas y herramientas que tienes como ingeniero de software te servirán también para construir aplicaciones de IA.



# Prompts

Los **prompts** son el último de los tres principales primitivos que proveen los servidores MCP. Están diseñados para ser controlados por el usuario de la aplicación, quien debería poder seleccionar qué prompt o prompts desea usar, similar a la interfaz que viste en el ejemplo de la aplicación host en la sección anterior.  

Cada definición de prompt tiene:  
- Un identificador único `name`.  
- Una descripción opcional legible por humanos.  
- Argumentos opcionales: una lista de diccionarios con:
  - `name`
  - descripción legible opcional
  - booleano `required` opcional indicando si el argumento es obligatorio.  

Corresponde a la implementación del cliente o la aplicación host que use el prompt inyectar valores para cada uno de los argumentos. Estos se llaman **prompts dinámicos**.

---

## Manejo de prompts con el SDK de Python

El SDK de Python simplifica el manejo de prompts mediante:  

- `Session.list_prompts()`: devuelve una lista de objetos `Prompt` con las propiedades mencionadas.  
- `get_prompt()`: permite obtener un prompt específico proporcionando su `name` y argumentos, devolviendo una lista de objetos `PromptMessage` listos para usar.  

Los **PromptMessage** pueden convertirse en diccionarios simples y enviarse directamente al LLM. Cada mensaje enviado al LLM tiene:  
- Claves `role` y `content` (requeridas).  
- `content` puede contener:
  - `TextContent`
  - `ImageContent`
  - `AudioContent`
  - `EmbeddedResource`  

Los **EmbeddedResource** son recursos incrustados dentro del bloque de contenido del mensaje, tal como en el ejemplo de la aplicación host de la sección anterior.

---

## Ejemplo de cliente MCP para prompts

```python
# client.py
from mcp.types import Prompt

class MCPClient:
    ...
    async def get_available_prompts(self) -> list[Prompt]:
        if not self._connected:
            raise RuntimeError("Client not connected to a server")

        prompt_result = await self._session.list_prompts()
        if not prompt_result.prompts:
            logger.warning("No prompts found on server")
        return prompt_result.prompts
````

Este patrón es similar al de recursos y herramientas:

1. Se verifica la conexión.
2. Se obtiene la lista de prompts del servidor.
3. Se asegura que la lista no esté vacía antes de devolverla al usuario.

El wrapper `get_prompt()` funciona de forma similar: se obtiene el nombre y los argumentos del usuario, se carga el prompt desde el servidor y se devuelve el objeto `PromptMessage` al cliente.

```


In [14]:
# client.py

from mcp.types import PromptMessage

class MCPClient:
    ...
    async def load_prompt(
        self, name: str, arguments: dict[str, str]
    ) -> list[PromptMessage]:
        if not self._connected:
            raise RuntimeError("Client not connected to a server")
        prompt_load_result = await self._session.get_prompt(name=name, arguments=arguments)

        if not prompt_load_result.messages:
            logger.warning(f"No prompt found for prompt {name}")
        else:
            logger.warning(f"Loaded prompt {name} with description {prompt_load_result.description}")
        return prompt_load_result.messages





* Esta función es similar a cómo se recuperan los recursos.
* Incluye controles estándar de conexión y pasa el `name` y `arguments` directamente al método `get_prompt()` de la sesión.
* Se agrega logging para visibilidad.

**Nota sobre notificaciones:**

* El protocolo MCP permite notificaciones cuando cambia la lista de prompts disponibles.
* Actualmente, no hay API de usuario para manejar estas notificaciones, aunque los clientes pueden suscribirse a ellas.
* Para usarlas, habría que enrutar las notificaciones a un callback propio, siguiendo ejemplos en el SDK (como `Session._received_request()`) o creando una función que maneje todos los mensajes y pasándola al constructor del cliente en `message_handler`.

---

# Integración en la aplicación host

* Se reutiliza la misma base de aplicación que en los ejemplos anteriores.
* Se obtiene la lista de prompts disponibles.
* Se muestran al usuario para que seleccione cuál usar.
* El prompt seleccionado se envía tal cual al LLM y se maneja la respuesta.

In [16]:

# ============================================================
# agent.py
# ============================================================

import json
import asyncio
from typing import Any
from pathlib import Path
from mcp.client import MCPClient
from anthropic import Anthropic

# ============================================================
# Clase principal del agente
# ============================================================
class Agent:
    def __init__(self, mcp_client: MCPClient, anthropic_client: Anthropic):
        self.mcp_client = mcp_client
        self.anthropic_client = anthropic_client
        self.available_resources = {}
        self.available_prompts = {}

    # ========================================================
    # Selección de recursos con LLM
    # ========================================================
    async def _select_resources(self, user_query: str) -> list[str]:
        """Usa el LLM para seleccionar recursos relevantes."""
        ...
    
    # ========================================================
    # Selección de prompts con LLM
    # ========================================================
    async def _select_prompts(self, user_query: str) -> list[dict[str, Any]]:
        """Usa el LLM para seleccionar prompts relevantes."""
        if not self.available_prompts:
            return []

        prompts = [prompt.model_dump_json() for prompt in self.available_prompts.values()]

        selection_prompt = f"""
        Given this user question: "{user_query}"
        
        And these available prompt templates:
        {json.dumps(prompts, indent=2)}
        
        Which prompts (if any) would provide helpful instructions or guidance for answering this question?
        Return a JSON array of prompt objects which have a name (string) and arguments (objects where the
        keys are the named parameter name and value is the argument value), or an empty array if no prompts
        are needed. Only include prompts that are directly relevant.
        
        Example: [{{"name": "calculation-helper", "arguments": {{"operation": "addition"}}}},
                 {{"name": "step-by-step-math", "arguments": {{}}}}] or []
        """

        try:
            response = self.anthropic_client.messages.create(
                max_tokens=200,
                messages=[{"role": "user", "content": selection_prompt}],
                model="claude-sonnet-4-0",
            )

            response_text = response.content[0].text.strip()
            if "[" in response_text and "]" in response_text:
                start = response_text.find("[")
                end = response_text.rfind("]") + 1
                json_part = response_text[start:end]
                selected_prompts = json.loads(json_part)
                return [p for p in selected_prompts if p["name"] in self.available_prompts]

        except Exception as e:
            logger.warning(f"Failed to select prompts with LLM: {e}")

        return []

    # ========================================================
    # Carga de recursos seleccionados
    # ========================================================
    async def _load_selected_resources(self, resource_names: list[str]) -> list[dict[str, Any]]:
        """Carga los recursos seleccionados (pendiente de implementación)."""
        ...

    # ========================================================
    # Carga de prompts seleccionados
    # ========================================================
    async def _load_selected_prompts(self, prompts: list[dict[str, Any]]) -> str:
        """Carga los prompts seleccionados como instrucciones del sistema."""
        system_instructions = []

        for prompt in prompts:
            if prompt["name"] in self.available_prompts:
                print(f"Using prompt: {prompt['name']}")
                try:
                    prompt_content = await self.mcp_client.load_prompt(
                        name=prompt["name"], arguments=prompt["arguments"]
                    )

                    # Extraer el texto del prompt
                    prompt_text = ""
                    for message in prompt_content:
                        if hasattr(message.content, "text"):
                            prompt_text += message.content.text + "\n"
                        elif isinstance(message.content, str):
                            prompt_text += message.content + "\n"

                    if prompt_text.strip():
                        system_instructions.append(
                            f"[Prompt: {prompt['name']}]\n{prompt_text.strip()}"
                        )

                except Exception as e:
                    print(f"Error loading prompt {prompt['name']}: {e}")

        return "\n\n".join(system_instructions)

    # ========================================================
    # Actualización de recursos y prompts disponibles
    # ========================================================
    async def _refresh(self) -> None:
        """Obtiene y actualiza los recursos y prompts disponibles del cliente MCP."""
        available_resources = await self.mcp_client.get_available_resources()
        self.available_resources = {resource.name: resource for resource in available_resources}

        available_prompts = await self.mcp_client.get_available_prompts()
        self.available_prompts = {prompt.name: prompt for prompt in available_prompts}

    # ========================================================
    # Bucle principal de ejecución del agente
    # ========================================================
    async def run(self):
        """Ejecuta el agente en modo interactivo."""
        try:
            print(
                "Welcome to your AI Assistant. Type 'goodbye' to quit or 'refresh' to reload and redisplay available resources."
            )
            await self.mcp_client.connect()
            available_tools = await self.mcp_client.get_available_tools()
            await self._refresh()

            print(
                f"Loaded {len(self.available_resources)} resources and {len(self.available_prompts)} prompts"
            )

            while True:
                prompt = input("You: ")

                if prompt.lower() == "goodbye":
                    print("AI Assistant: Goodbye!")
                    break

                if prompt.lower() == "refresh":
                    await self._refresh()
                    continue

                # ===============================================
                # Selección y carga de recursos y prompts
                # ===============================================
                selected_resource_names = await self._select_resources(prompt)
                selected_prompt_names = await self._select_prompts(prompt)

                context_messages = await self._load_selected_resources(selected_resource_names)
                system_instructions = await self._load_selected_prompts(selected_prompt_names)

                # ===============================================
                # Construcción de la conversación
                # ===============================================
                user_content = [{"type": "text", "text": prompt}]
                if context_messages:
                    user_content.extend(context_messages)

                conversation_messages = [{"role": "user", "content": user_content}]

                # ===============================================
                # Bucle de interacción con el modelo y herramientas
                # ===============================================
                while True:
                    create_message_args = {
                        "max_tokens": 4096,
                        "messages": conversation_messages,
                        "model": "claude-sonnet-4-0",
                        "tools": available_tools,
                        "tool_choice": {"type": "auto"},
                    }

                    if system_instructions:
                        create_message_args["system"] = system_instructions

                    current_response = self.anthropic_client.messages.create(**create_message_args)

                    # Agregar respuesta del asistente a la conversación
                    conversation_messages.append({"role": "assistant", "content": current_response.content})

                    # ===============================================
                    # Manejo de uso de herramientas
                    # ===============================================
                    if current_response.stop_reason == "tool_use":
                        tool_use_blocks = [block for block in current_response.content if block.type == "tool_use"]

                        tool_results = []
                        for tool_use in tool_use_blocks:
                            print(f"Using tool: {tool_use.name}")
                            tool_result = await self.mcp_client.use_tool(
                                tool_name=tool_use.name,
                                arguments=tool_use.input
                            )
                            tool_results.append(
                                {
                                    "type": "tool_result",
                                    "tool_use_id": tool_use.id,
                                    "content": "\n".join(tool_result),
                                }
                            )

                        conversation_messages.append({"role": "user", "content": tool_results})
                        continue

                    # ===============================================
                    # Respuesta final del asistente
                    # ===============================================
                    else:
                        text_blocks = [
                            content.text for content in current_response.content
                            if hasattr(content, "text") and content.text.strip()
                        ]

                        if text_blocks:
                            print(f"Assistant: {text_blocks[0]}")
                        else:
                            print("Assistant: [No text response available]")

                        break
        finally:
            await self.mcp_client.disconnect()



ImportError: cannot import name 'MCPClient' from 'mcp.client' (C:\Users\mcssa\.conda\envs\google-adk\Lib\site-packages\mcp\client\__init__.py)

# Integración de Prompts en el Agente

Esta versión del código realiza todas las acciones descritas en la introducción del ejemplo.

### 1. Bucle principal e inicialización
En los bucles principales del agente, primero se llama a la función `_refresh()`, la cual invoca el método `get_available_prompts()` que implementamos en el cliente.  
Esta función llena el atributo `available_prompts` con un diccionario similar al del ejemplo de **Resources**, donde:
- Las **claves** son los nombres de los prompts.  
- Los **valores** son los objetos `Prompt`.

### 2. Selección de prompts relevantes
Dentro del bucle del agente se llama a `_select_prompts()` para que el modelo LLM seleccione los prompts más relevantes para la consulta del usuario, de forma similar al ejemplo de **Resources**, pero con algunas diferencias:
- Cada objeto `Prompt` se convierte en una cadena JSON usando `model_dump_json()`.  
- Esto permite que el prompt de selección incluya toda la información del objeto (descripción y argumentos), ayudando al modelo a decidir cuáles prompts son los más adecuados.

### 3. Carga y validación de prompts seleccionados
Luego se llama a `_load_selected_prompts()` para:
- Filtrar los prompts alucinados (no válidos).  
- Obtener los prompts restantes mediante el cliente.  
- Transformarlos en el formato de **prompts del sistema** que utiliza el LLM.

### 4. Preparación de la llamada al LLM
Antes de llamar al modelo LLM:
1. Se crea un diccionario con los argumentos del mensaje.  
2. Si existen prompts cargados, se añaden bajo la clave `system`.  
3. Finalmente, estos argumentos se desempaquetan en la llamada `Anthropic.create()`.

### 5. Resto del agente
El resto del agente conserva la misma lógica del ejemplo anterior, incluyendo:
- La interacción con el usuario.  
- El uso de herramientas.  
- La construcción del historial de conversación.
